<a href="https://colab.research.google.com/github/Alih111/naivee-and-logistic-nlp/blob/main/NLP_naivebayes_logisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
from datasets import load_dataset
sst_dataset=load_dataset("sst")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for sst contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/sst
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/8544 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1101 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2210 [00:00<?, ? examples/s]

getting training data sentences and labels


In [ ]:
train_data = sst_dataset["train"]
sentences=train_data["sentence"]
test_data = sst_dataset["test"]
test_sen=test_data['sentence']
test_score=test_data['label']
validate=sst_dataset["validation"]
validate_sen=validate['sentence']
validate_score=validate['label']
sentences=[s.lower() for s in sentences]
test_sen=[s.lower() for s in test_sen]
validate_sen=[s.lower() for s in validate_sen]
len(sentences)

8544

classify sentences according to score

•From 0 to 0.2 (0.2 included) will be class 0 “very negative”.
•From 0.2 to 0.4 (0.4 included) will be class 1 “negative”.
•From 0.4 to 0.6 (0.6included)will be class 2 “neutral”.•From 0.6to 0.8 (0.8 included)will be class 3 “positive”.•From 0.8to 1.0(1.0included)will be class 4 “very positive”.

In [ ]:
import numpy as np

scores= train_data["label"]
classes = [[] for _ in range(5)]
#put each sentence in the class that it belongs to
i=0
for score in scores:

    if score <= 0.2:
      scores[i]=int(0)
      classes[0].append(sentences[i])
    elif score > 0.2 and score <= 0.4:
      scores[i]=int(1)
      classes[1].append(sentences[i])
    elif score > 0.4 and score <=0.6:
      scores[i]=int(2)
      classes[2].append(sentences[i])
    elif score > 0.6 and score <=0.8:
      scores[i]=int(3)
      classes[3].append(sentences[i])
    elif score > 0.8 and score <=1:
      scores[i]=(4)
      classes[4].append(sentences[i])
    i+=1

i=0
for score in test_score:

    if score <= 0.2:
      test_score[i]=int(0)
    elif score > 0.2 and score <= 0.4:
      test_score[i]=int(1)
    elif score > 0.4 and score <=0.6:
      test_score[i]=int(2)
    elif score > 0.6 and score <=0.8:
      test_score[i]=int(3)
    elif score > 0.8 and score <=1:
      test_score[i]=int(4)
    i+=1


i=0
for score in validate_score:

    if score <= 0.2:
      validate_score[i]=0
    elif score > 0.2 and score <= 0.4:
      validate_score[i]=1
    elif score > 0.4 and score <=0.6:
      validate_score[i]=2
    elif score > 0.6 and score <=0.8:
      validate_score[i]=3
    elif score > 0.8 and score <=1:
      validate_score[i]=4
    i+=1


print(len(test_score))

2210


count number of occurences of all words in the document


In [ ]:
from collections import Counter

sentences_np=np.array(sentences)
all_words = ' '.join(sentences_np).split()
print(len(all_words))
word_counts = Counter(all_words)
word_counts_dict = dict(word_counts)
del word_counts
del sentences_np

print(len(word_counts_dict))

163566
16579


naivee bayes

In [ ]:
def countWord(goal,words):#count words in a single class
    return  np.sum(words == goal)

naive bayes training loop


In [ ]:
from math import log

words_probablites={0: {},#will store the likelihood of each word in each class
                  1: {},
                   2: {},
                   3: {},4: {}}
Ndoc = sum(len(row) for row in classes)# counts the total number of words
stopWords=[]
priors=[]
priors_sk=[]
m=0
for k in range(len(classes)):
  Nc=len(classes[k])#count total number of words in each class
  print(Nc)
  priors.append(np.float16(np.log2(Nc/Ndoc)))
  myclass = ' '.join(classes[k]).split()#get vocubulary for each class
  print(myclass)
  #myclass = [word for word in myclass if word not in stopWords ]

  word_counts = Counter(myclass)

  this_word_counts_dict = dict(word_counts)#stores words count in each class to measure likelihood
  print(this_word_counts_dict)
  for word in word_counts_dict:
      if word not in words_probablites[k] :#and len(word) > 1
        #occurence=countWord(word,words)
        if word in this_word_counts_dict:
          log_likelihood = np.float16(np.log2((this_word_counts_dict[word] + 1) / (len(word_counts) + 1)))
        else:
          log_likelihood = np.float16(np.log2(( 1) / (len(word_counts)+ 1)))
        words_probablites[k][word]=log_likelihood

  del this_word_counts_dict



1092
['...', 'a', 'sour', 'little', 'movie', 'at', 'its', 'core', ';', 'an', 'exploration', 'of', 'the', 'emptiness', 'that', 'underlay', 'the', 'relentless', 'gaiety', 'of', 'the', '1920', "'s", '...', 'the', 'film', "'s", 'ending', 'has', 'a', '``', 'what', 'was', 'it', 'all', 'for', '?', "''", 'as', 'it', 'is', ',', 'it', "'s", 'too', 'long', 'and', 'unfocused', '.', 'a', 'decided', 'lack', 'of', 'spontaneity', 'in', 'its', 'execution', 'and', 'a', 'dearth', 'of', 'real', 'poignancy', 'in', 'its', 'epiphanies', '.', 'you', "'ll", 'forget', 'about', 'it', 'by', 'monday', ',', 'though', ',', 'and', 'if', 'they', "'re", 'old', 'enough', 'to', 'have', 'developed', 'some', 'taste', ',', 'so', 'will', 'your', 'kids', '.', 'the', 'ya-ya', "'s", 'have', 'many', 'secrets', 'and', 'one', 'is', '-', 'the', 'books', 'are', 'better', '.', 'wedding', 'feels', 'a', 'bit', 'anachronistic', '.', 'the', 'paradiso', "'s", 'rusted-out', 'ruin', 'and', 'ultimate', 'collapse', 'during', 'the', 'film', "'

test naivee bayes loop



In [ ]:
print(priors)
final_sum= [[] for _ in range(5)] #will contian the probability of each sentence to be in each class
j=0
for i in range (len(classes)):
  for sentence in test_sen:
      sum1=priors[i]
      words=sentence.split(' ')
      for word in words:#measures the probabily of each sentence for a specific class
        #if len(word) > 1:#skip one letter words
          if word in words_probablites[i] :#check if word appeared before in this class
            sum1+=words_probablites[i][word]
      final_sum[i].append(sum1)

  j+=1
final_sum=np.array(final_sum)
print(final_sum[:,0])
max_indices = np.argmax(final_sum, axis=0)
del words_probablites
del final_sum
max(max_indices)



[-2.969, -1.945, -2.395, -1.88, -2.73]
[-33.25 -28.78 -27.4  -26.94 -31.53]


4



```
# This is formatted as code
```

sklearn naivee bayes Method 1



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
def simple_token(text):
  return text.split()
model=Pipeline([('vectorizer',CountVectorizer(tokenizer=simple_token,lowercase=True)),('classifier',MultinomialNB(alpha=1))])
model.fit(sentences,scores)
labels = model.predict(test_sen)
labels[0]



/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


3

## sklearn naivee bayes Method 2

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

vectorizer = CountVectorizer(analyzer='word', ngram_range=(2, 2))
X_train = vectorizer.fit_transform(sentences)
X_test = vectorizer.transform(test_sen)
# X_val = vectorizer.transform(validate_sen).astype(int)
# print(vectorizer.get_feature_names_out())
# print(X_train.shape)
# print(X_test.shape)
# print(X_val.shape)



naive_bayes = MultinomialNB()
naive_bayes.fit(X_train, scores)
labels = naive_bayes.predict(X_test)
labels

array([1, 3, 3, ..., 4, 1, 3])

confusion matrix

In [ ]:

class ConfusionMatrix:
  def __init__(self,actual,predicted):
    self.confMat=np.zeros((5,5))
    self.true=0
    self.false=0
    self.t0=0
    for k in range(len(actual)):#predicted row and actual column
      j=predicted[k]#predicted label

      i=actual[k]#actual label
      self.confMat[i][j]+=1
      if i == j:
        self.true+=1
      else:
        self.false+=1
  def accuracy(self):
    return self.true/(self.true+self.false)


  def precision(self):
    precisions = []
    for i in range(len(self.confMat)):
        true_positives = self.confMat[i][i]
        false_positives = sum(self.confMat[j][i] for j in range(len(self.confMat)) if j != i)
        if true_positives + false_positives == 0:
            precisions.append(0)
        else:
            precisions.append(true_positives / (true_positives + false_positives))
    return precisions
  def recall(self):
    recall = []
    for i in range(len(self.confMat)):
        true_positives = self.confMat[i][i]
        false_negative = sum(self.confMat[i,j] for j in range(len(self.confMat)) if j != i)
        if true_positives + false_negative == 0:
            recall.append(0)
        else:
            recall.append(true_positives / (true_positives + false_negative))
    return recall
  def F1Score(self):
    precision = np.array(self.precision())
    recall = np.array(self.recall())
    f1_scores = []
    for i in range(len(precision)):
        if precision[i] + recall[i] == 0:
            f1_scores.append(0)  # Handle the case of both precision and recall being zero
        else:
            f1_scores.append(2 * precision[i] * recall[i] / (precision[i] + recall[i]))
    return f1_scores
  def percisionMacro(self):
    precision = np.array(self.precision())
    return sum(precision)/5
  def recallMacro(self):
    recall = np.array(self.recall())
    return sum(recall)/5
  def f1Macro(self):
    F1Score = (self.F1Score())
    return sum(F1Score)/5

sklearnMethod=ConfusionMatrix(actual=test_score,predicted=labels)
myMethod=ConfusionMatrix(actual=test_score,predicted=max_indices)
print(sklearnMethod.confMat)
print("/////////////////")
print(myMethod.confMat)
print(f"my method precision = {myMethod.precision()} sklearnprecision = {sklearnMethod.precision()}")
print(f"my method recall = {myMethod.recall()} sklearn recall = {sklearnMethod.recall()}")
print(f"my method f1 = {myMethod.F1Score()} sklearn F1 = {sklearnMethod.F1Score()}")
print(f"my method p m = {myMethod.percisionMacro()} sklearn p m = {sklearnMethod.percisionMacro()}")
print(f"my method r m = {myMethod.recallMacro()} sklearn r ,m = {sklearnMethod.recallMacro()}")
print(f"my method f m = {myMethod.f1Macro()} sklearn f1 Macro = {sklearnMethod.f1Macro()}")
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score


[[ 34. 146.  20.  72.   7.]
 [ 34. 332.  73. 183.  11.]
 [ 12. 149.  38. 180.  10.]
 [  4. 127.  31. 310.  38.]
 [  5.  64.  33. 218.  79.]]
/////////////////
[[ 31. 184.   2.  58.   4.]
 [ 29. 376.  18. 191.  19.]
 [ 10. 155.  10. 199.  15.]
 [  4.  63.   6. 376.  61.]
 [  2.  29.   4. 254. 110.]]
my method precision = [0.40789473684210525, 0.46592317224287483, 0.25, 0.34879406307977734, 0.5263157894736842] sklearnprecision = [0.38202247191011235, 0.4058679706601467, 0.19487179487179487, 0.32191069574247144, 0.5448275862068965]
my method recall = [0.1111111111111111, 0.593996840442338, 0.02570694087403599, 0.7372549019607844, 0.2756892230576441] sklearn recall = [0.12186379928315412, 0.5244865718799369, 0.09768637532133675, 0.6078431372549019, 0.19799498746867167]
my method f1 = [0.17464788732394365, 0.5222222222222223, 0.046620046620046623, 0.47355163727959704, 0.3618421052631579] sklearn F1 = [0.18478260869565216, 0.45761543762922124, 0.13013698630136986, 0.42090970807875083, 0.2904

## Logistic Regression

Feature Representation

In [ ]:
from ctypes import BigEndianStructure
def generate_word_bi_grams(sentences):
    # Create a set to store unique word bi-grams
    unique_bi_grams =set()

    # Iterate through each sentence to extract word bi-grams
    for sentence in sentences:
        words = sentence.split()
       # words=[word for word in words if word.is]
        # words= [word for word in words if word not in stopWords ]
        bi_grams = zip(words, words[1:])
        unique_bi_grams.update(bi_grams)

    # Convert the set of unique word bi-grams to a sorted list

    return sorted(list(unique_bi_grams))

In [ ]:
bi_grams = generate_word_bi_grams(sentences)

print(bi_grams)

[('!', "'"), ('#', '3'), ('#', '9'), ('$', '1.8'), ('$', '20'), ('$', '40'), ('$', '50-million'), ('$', '8'), ('$', '9'), ('$', '99'), ('&', 'about'), ('&', 'augmented'), ('&', 'christian'), ('&', 'co.'), ('&', 'heart-rate-raising'), ('&', 'j'), ('&', 'paulette'), ('&', 'stitch'), ('&', 'tonto'), ('&', 'w'), ('&', 'z-boys'), ("'", '('), ("'", ')'), ("'", ','), ("'", '-'), ("'", '--'), ("'", '.'), ("'", '..'), ("'", '...'), ("'", '20,000'), ("'", ':'), ("'", ';'), ("'", '`'), ("'", 'a'), ("'", 'about'), ("'", 'according'), ("'", 'achievement'), ("'", 'actors'), ("'", 'advises'), ("'", 'affections'), ("'", 'ambitious'), ("'", 'amusing'), ("'", 'anarchy'), ("'", 'and'), ("'", 'anderson'), ("'", 'anguish'), ("'", 'anti-'), ("'", 'appeared'), ("'", 'applies'), ("'", 'are'), ("'", 'as'), ("'", 'at'), ("'", 'award'), ("'", 'based'), ("'", 'because'), ("'", 'best'), ("'", 'better'), ("'", 'big-screen'), ("'", 'bohemian'), ("'", 'book'), ("'", 'borscht'), ("'", 'bow'), ("'", 'but'), ("'", 'butl

In [ ]:
len(bi_grams)

80936

In [ ]:
def generate_sparse_feature_representation(sentences, unique_bi_grams):
    feature_representation = []
    # Iterate through each sentence to create the sparse feature vector
    i=0
    for sentence in sentences:
        words = sentence.split()
        bi_grams = set(zip(words, words[1:]))

        # Create a binary vector for the current sentence
        sentence_vector=[]
        for bi_gram in bi_grams:
          if bi_gram in unique_bi_grams:
            index = unique_bi_grams.index(bi_gram)
            sentence_vector.append(index)
        feature_representation.append(sentence_vector)
        i+=1

    return feature_representation

In [ ]:
features = generate_sparse_feature_representation(sentences, bi_grams)

In [ ]:
# each sentence has a vector of features of length
len(features)
# len(test_sen)
# features[2]

8544

sigmoid Function


In [ ]:
def sigmoid(x):
  return (1/(1+np.exp(-x)))
# turn scores to softmax matrix
def turnScores(scores):
  new_scores=np.zeros((len(scores),5))
  i=0
  for score in scores:
    new_scores[i,score]=1
    i+=1
  return new_scores
def predict(weights,bias,x):
  z=np.dot(weights,x)

In [ ]:
# z = WX+B
b=np.zeros((5))#bias for each sentence
new_scores=turnScores(scores)
print(new_scores[0])
W=np.zeros((5,len(features[0])))
W.shape

[0. 0. 0. 1. 0.]


(5, 34)

In [ ]:
def multiClassLogistic(scores,features,learningRate,num_epochs,sample_size,num_of_samples):
  b=np.zeros((5))#bias for each sentence
  W=np.zeros((5,len(features[0])+1))
  for epoch in range(num_epochs):
      random_samples = np.random.uniform(0, len(features), 64)
      for sample in random_samples:
        X=features[:,sample]
        Y=scores[:,sample]



## IMplementation

In [ ]:
def turnScores(scores):
  new_scores=np.zeros((len(scores),5))
  i=0
  for score in scores:
    new_scores[i,score]=1
    i+=1
  return new_scores


y_one_hot = turnScores(scores)


In [ ]:
y_one_hot.shape

In [ ]:
num_features = len(bi_grams)



In [ ]:
def softmax(logits):
    exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))
    return exp_logits / np.sum(exp_logits, axis=1, keepdims=True)


In [ ]:
def forward_pass(X, weights, bias):
    logits = np.dot(X, weights) + bias
    probabilities = softmax(logits)
    return probabilities


In [ ]:
def cross_entropy_loss(y_true, probabilities):
    return -np.sum(y_true * np.log(probabilities + 1e-10))


In [ ]:
def backward_pass(X, y_true, probabilities, weights, bias, learning_rate):
    error = probabilities - y_true#probablites is y hat
    grad_weights = np.dot(X.T, error)#dl/dw
    grad_bias = np.sum(error, axis=0, keepdims=True)#dl/db

    weights -= learning_rate * grad_weights
    bias -= learning_rate * grad_bias

    return weights, bias


In [ ]:
def generate_sparse_test(sentences, unique_bi_grams):
    # Initialize a list to store the feature representation for each sentence
    feature_representation = []
    # Iterate through each sentence to create the sparse feature vector
    print('a')
    for sentence in sentences:
        words = sentence.split()
        bi_grams = set(zip(words, words[1:]))

        # Create a binary vector for the current sentence
        sentence_vector = [1 if bi_gram in bi_grams else 0 for bi_gram in unique_bi_grams]
        feature_representation.append(sentence_vector)

    return feature_representation




In [ ]:
X_valid=generate_sparse_test(validate_sen,bi_grams)

X_valid=np.array(X_valid)
import random
num_epochs = 400
learning_rate = 0.1
weights = np.random.rand(num_features, 5)
bias = np.zeros((1, 5))
for epoch in range(num_epochs):
      X = []
      Y = []
      random_samples = random.sample(range(len(features)), 128)
      for sample in random_samples:
        sentence_vector = np.zeros(len(bi_grams))
        feature_list = features[sample]
        Y.append(y_one_hot[sample])
        for i in feature_list:
          sentence_vector[i] = 1
        X.append(sentence_vector)

      X_batch = np.array(X)
      y_batch = np.array(Y)

      # Forward pass
      probabilities = forward_pass(X_batch, weights, bias)
      if epoch > 150:
        learning_rate=0.05

      # Compute loss
      loss = cross_entropy_loss(y_batch, probabilities)

      # Backward pass (Gradient Descent)
      weights, bias = backward_pass(X_batch, y_batch, probabilities, weights, bias, learning_rate)
      y_valid = np.argmax(forward_pass(X_valid, weights, bias),axis=1)
      myMethod=ConfusionMatrix(actual=validate_score,predicted=y_valid)
      a=myMethod.accuracy()



      print(f'Epoch {epoch + 1}, Loss: {loss}')


a
Epoch 1, Loss: 260.33159464953604
Epoch 2, Loss: 267.2745868875181
Epoch 3, Loss: 305.75558165526934
Epoch 4, Loss: 292.34410579429385
Epoch 5, Loss: 344.9419939394703
Epoch 6, Loss: 357.12774884026317
Epoch 7, Loss: 288.4365370862913
Epoch 8, Loss: 299.07928872539924
Epoch 9, Loss: 271.57049332625195
Epoch 10, Loss: 271.683123042542
Epoch 11, Loss: 289.4467582490571
Epoch 12, Loss: 255.3139798875273
Epoch 13, Loss: 256.9959992459595
Epoch 14, Loss: 284.8928279417211
Epoch 15, Loss: 369.8150613754849
Epoch 16, Loss: 260.40823210400333
Epoch 17, Loss: 246.0204605908039
Epoch 18, Loss: 245.61075381726982
Epoch 19, Loss: 312.7471533059877
Epoch 20, Loss: 263.0452061568247
Epoch 21, Loss: 311.7691875687646
Epoch 22, Loss: 271.5988065903374
Epoch 23, Loss: 259.7806126943905
Epoch 24, Loss: 267.19233376740243
Epoch 25, Loss: 280.0341929752779
Epoch 26, Loss: 345.2220252409884
Epoch 27, Loss: 197.54751006518495
Epoch 28, Loss: 232.7795205979251
Epoch 29, Loss: 280.36053984228
Epoch 30, Loss

In [ ]:
#change hyper parameteres

In [ ]:
X_test=generate_sparse_test(test_sen,bi_grams)
X_test= np.array(X_test)


a


In [ ]:
y_test = np.argmax(forward_pass(X_test, weights, bias),axis=1)
myMethod=ConfusionMatrix(actual=test_score,predicted=y_test)
print(myMethod.confMat)
print(f"precision = {myMethod.precision()}")
print(f"recall = {myMethod.recall()}")
print(f"f1 score= {myMethod.F1Score()}")
print(f"precision macro = {myMethod.percisionMacro()}")
print(f"recall macro = {myMethod.recallMacro()}")
print(f"f1 macro = {myMethod.f1Macro()}")


[[ 45.  51.  27.  12.  22.]
 [132. 295. 166. 130.  83.]
 [ 40. 126.  63.  95.  54.]
 [ 44. 126. 107. 221. 143.]
 [ 18.  35.  26.  52.  97.]]
precision = [0.16129032258064516, 0.4660347551342812, 0.16195372750642673, 0.43333333333333335, 0.24310776942355888]
recall = [0.28662420382165604, 0.36600496277915634, 0.16666666666666666, 0.34477379095163807, 0.42543859649122806]
f1 score= [0.2064220183486238, 0.41000694927032666, 0.16427640156453713, 0.38401390095569077, 0.30940988835725675]
precision macro = 0.29314398159564903
recall macro = 0.317901644142069
f1 macro = 0.294825831699287


## Sklearn Logistic regression


In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

X_train=generate_sparse_test(sentences,set(bi_grams))




classifier = SGDClassifier(loss='log_loss', max_iter=1000, random_state=42)

classifier.fit(X_train, scores)
del X_train
X_test=generate_sparse_test(test_sen,set(bi_grams))
X_test= np.array(X_test)
y_pred = classifier.predict(X_test)
del X_test
myMethod=ConfusionMatrix(actual=test_score,predicted=y_pred)
print(myMethod.confMat)
print(f"precision = {myMethod.precision()}")
print(f"recall = {myMethod.recall()}")
print(f"f1 score= {myMethod.F1Score()}")
print(f"precision macro = {myMethod.percisionMacro()}")
print(f"recall macro = {myMethod.recallMacro()}")
print(f"f1 macro = {myMethod.f1Macro()}")
# this is the one that comparble to my implementation because it use stochastic gradient as mine

a
a
[[ 36.  46.  18.   9.   7.]
 [114. 261. 113.  77.  40.]
 [ 40. 123.  64.  75.  39.]
 [ 74. 176. 174. 280. 196.]
 [ 15.  27.  20.  69. 117.]]
precision = [0.12903225806451613, 0.41232227488151657, 0.16452442159383032, 0.5490196078431373, 0.2932330827067669]
recall = [0.3103448275862069, 0.43140495867768597, 0.187683284457478, 0.3111111111111111, 0.4717741935483871]
f1 score= [0.18227848101265823, 0.42164781906300486, 0.17534246575342463, 0.3971631205673759, 0.3616692426584235]
precision macro = 0.30962632901795345
recall macro = 0.34246367507617387
f1 macro = 0.30762022581097737


In [ ]:
from sklearn.linear_model import LogisticRegression

X_train=generate_sparse_test(sentences,set(bi_grams))

#this is better because this solver is better for sparse features
#the solver here is diffrent not sgd

classifier = LogisticRegression(max_iter=1000)


classifier.fit(X_train, scores)

del X_train
X_test=generate_sparse_test(test_sen,set(bi_grams))
X_test= np.array(X_test)

y_pred = classifier.predict(X_test)
del X_test
myMethod=ConfusionMatrix(actual=test_score,predicted=y_pred)
print(myMethod.confMat)
print(f"precision = {myMethod.precision()}")
print(f"recall = {myMethod.recall()}")
print(f"f1 score= {myMethod.F1Score()}")
print(f"precision macro = {myMethod.percisionMacro()}")
print(f"recall macro = {myMethod.recallMacro()}")
print(f"f1 macro = {myMethod.f1Macro()}")


a
a
[[ 32.  35.  18.   7.   6.]
 [143. 323. 153. 104.  65.]
 [ 33.  99.  57.  68.  38.]
 [ 61. 156. 146. 274. 189.]
 [ 10.  20.  15.  57. 101.]]
precision = [0.11469534050179211, 0.5102685624012638, 0.14652956298200515, 0.5372549019607843, 0.2531328320802005]
recall = [0.32653061224489793, 0.4098984771573604, 0.19322033898305085, 0.33171912832929784, 0.4975369458128079]
f1 score= [0.16976127320954906, 0.4546094299788881, 0.16666666666666669, 0.4101796407185629, 0.3355481727574751]
precision macro = 0.31237623998520914
recall macro = 0.35178110050548295
f1 macro = 0.3073530366662284


In [ ]:
del myMethod